In [4]:
import pandas as pd
import locale
from dateutil import parser

df = pd.read_csv('histo_trafic.csv', sep=';')

df = df.dropna(how='all')

locale_configured = False
for loc in ['fr_FR.UTF-8', 'fr_FR', 'French_France.1252', 'fra']:
    try:
        locale.setlocale(locale.LC_TIME, loc)
        locale_configured = True
        break
    except:
        continue


df['tstamp_original'] = df['tstamp'].copy()


# Changement du format des dates

def parse_french_date(date_str):
    if pd.isna(date_str) or date_str == '':
        return pd.NaT
    
    date_str = ' '.join(date_str.split())  
    
    mois_fr = {
        'janvier': '01', 'février': '02', 'mars': '03', 'avril': '04',
        'mai': '05', 'juin': '06', 'juillet': '07', 'août': '08',
        'septembre': '09', 'octobre': '10', 'novembre': '11', 'décembre': '12'
    }
    
    # Format "lundi 6 août 2018"
    try:
        parts = date_str.split()
        if len(parts) == 4:
            jour = parts[1]
            mois = parts[2].lower()
            annee = parts[3]
            
            if mois in mois_fr:
                date_formatee = f"{jour.zfill(2)}/{mois_fr[mois]}/{annee}"
                return pd.to_datetime(date_formatee, format='%d/%m/%Y')
    except:
        pass
    
    try:
        return pd.to_datetime(date_str, format='%A %d %B %Y')
    except:
        pass
    
    try:
        return parser.parse(date_str, dayfirst=True)
    except:
        return pd.NaT



# Conversion des dates

df['tstamp'] = df['tstamp_original'].apply(parse_french_date)


df["trafic_mbps"] = (
    df["trafic_mbps"]
    .astype(str)
    .str.replace(",", ".", regex=False)                   
    .str.replace(r"[^0-9\.\-eE]", "", regex=True)          
)

df["trafic_mbps"] = pd.to_numeric(df["trafic_mbps"], errors="coerce")


# Sort by secteur and time

df = df.sort_values(['secteur', 'tstamp'], na_position='last').reset_index(drop=True)

# Conversion du format 5 min

date_debut = pd.Timestamp('2025-01-01 00:00:00')

def ajouter_dates_5min(group):
    group = group.reset_index(drop=True)
    group['dates_5min'] = [date_debut + pd.Timedelta(minutes=5*i) for i in range(len(group))]
    return group

df = df.groupby('secteur', group_keys=False).apply(ajouter_dates_5min).reset_index(drop=True)


df_export = df.copy()

df_export['tstamp'] = df_export['tstamp'].dt.strftime('%d/%m/%Y')
df_export['dates_5min'] = df_export['dates_5min'].dt.strftime('%d/%m/%Y %H:%M')

colonnes_export = ['secteur', 'site', 'tstamp', 'trafic_mbps', 'dates_5min']
df_export = df_export[colonnes_export]


df_export.to_csv('histo_trafic_avec_dates_5min.csv', sep=';', index=False)

print("Fichier généré : histo_trafic_avec_dates_5min.csv")


C:\Users\hp\AppData\Local\Temp\ipykernel_27276\4043878027.py:90: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('secteur', group_keys=False).apply(ajouter_dates_5min).reset_index(drop=True)


Fichier généré : histo_trafic_avec_dates_5min.csv
